In [2]:
import openai
import json

import base64
import requests

import pandas as pd
import numpy as np

import uuid
import os

### Reading descriptions dataset

In [73]:

descrip_df = pd.DataFrame(columns=["Case_ID", "First_Party", "Second_Party", "Image_ID"])
descrip_df.head()

,Case_ID,First_Party,Second_Party,Image_ID


In [3]:
descrip_df = pd.read_csv("data\data_descriptions_all.csv")

In [191]:
descrip_df.head()

,Case_ID,First_Party,Second_Party,Image_ID
0,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود باتجاه التقاطع وكان الضوء الأخضر في ص...,كنت أقود في الطريق ولم ألاحظ أن الإشارة كانت ح...,image.png
1,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود عند التقاطع وكانت الإشارة خضراء لي، و...,كنت أقود وأتبع كل القواعد. وفجأة جاءت السيارة ...,image.png
2,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود وفقًا للقانون والإشارة كانت خضراء بال...,كنت أقود ولم ألاحظ أن الإشارة كانت خضراء لي ول...,image.png
3,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود باتجاه التقاطع وأعتقد أنني كنت من الم...,كنت أقود وألاحظ أن الإشارة كانت خضراء بالنسبة ...,image.png


### Starting connection with openai API

In [4]:
api_key = "sk-proj-UxsihIsXy5aq4Dq3G2DuT3BlbkFJwOAd6S4hvjcZN3nsCKvO"
openai.api_key = api_key

#### Get the descriptions by sending the image to chat-gpt 4o

In [5]:
#encoding function to encode the images before sending it to the API
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [ ]:
#get the responses from the API and store them in the responses list, and save each image path in a list as well
images_folder = "data\imgs"
responses = []
images_paths = []


for image_name in os.listdir(images_folder):
  #image path
  image_path = os.path.join(images_folder, image_name)
  #store the image in the images_paths list
  images_paths.append(image_path)

  #encoded image
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            #prompt
            "text": """[no prose]
                      [Output Only JSON]
                      I have car crash image I want you to take place as the effected parities, where your will write a description of the first party side, then write the other party description of their perspective. 
                      and do it in 4 scenarios 
                      1st: both parties say the truth.
                      2nd: both parties laying
                      3rd: first party is laying.
                      4th: second party is laying
                      (all in arabic)
                      
                      so end result will be in json format:
                      {
                        scenario_n:{

                          first_party:

                          second_party:
                        },
                      }
                      """
          },
          {
            #include the image
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    #number of tokens
    "max_tokens": 650
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  print(response.json())
  #store the response in responses list
  responses.append(response.json())
  
  print("---------------------I AM HERE---------------------")

  # #get the descriptions content from the response
  # response_dict = response.json()
  # #extract the content
  # content = response_dict['choices'][0]['message']['content']

  # #clean the content string then turn it into json 
  # content = content.replace("```json", "").replace("```", "").strip()
  # # content = content.replace('",', '"')
  # descrip_json = json.loads(content)

  # #Store the descriptions in the dataframe
  # case_id = uuid.uuid4()

  # for i in range(1, 5):
  #     data = {
  #         "Case_ID": case_id,
  #         "First_Party": descrip_json[f'scenario_{i}']['first_party'],
  #         "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
  #         "Image_ID": image_path
  #     }

  #     #create new row 
  #     new_row_df = pd.DataFrame([data])
  #     #add the new row to the descrip dataframe
  #     descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True)


  # descrip_df.head(20)

In [84]:
#Store the responses and images id's in the dataframe
for r, img in zip(responses, images_paths):
    #get the descriptions content from the response
    response_dict = r
    #extract the content
    content = response_dict['choices'][0]['message']['content']

    #clean the content string then turn it into json 
    content = content.replace("```json",'').replace("```",'').strip()
    # content = content.replace('",', '"')
    
    descrip_json = json.loads(content)

    if isinstance(descrip_json, list):
        continue

    #Store the descriptions in the dataframe
    case_id = uuid.uuid4()

    for i in range(1, 5):
        data = {
            "Case_ID": case_id,
            "First_Party": descrip_json[f'scenario_{i}']['first_party'],
            "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
            "Image_ID": img
        }

        #create new row 
        new_row_df = pd.DataFrame([data])
        #add the new row to the descrip dataframe
        descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True) 

In [8]:
# descrip_df.to_csv("data\descriptions.csv", index=False)

### Get the fault assessment based on the generated descriptions using chat-gpt 4o model